In [12]:
import nrrd
import numpy as np
import os
from skimage import measure
import csv

In [2]:
def create_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
    return None

def faces_to_edges(faces):
    edges = []
    for face in faces:
        edges.append(sorted([face[0],face[1]]))
        edges.append(sorted([face[1],face[2]]))
        edges.append(sorted([face[2],face[0]]))
    seen = set()
    unique_pairs = []
    
    for pair in edges:
        tup = tuple(pair)  # convert to tuple so it’s hashable
        if tup not in seen:
            seen.add(tup)
            unique_pairs.append(pair)
    return unique_pairs

In [3]:
segmentation_folder_path = '../segmentations/'

In [4]:
segmentation_file_name = 'Lung segmentation-10.seg.nrrd'

In [7]:
folder_path = './lung_segmentations'

In [8]:
file_name = 'segmentation-10'

In [6]:
data, header = nrrd.read( segmentation_folder_path + segmentation_file_name )
masks = {}
for i in range(data.shape[0]):
    seg_id = header.get(f"Segment{i}_ID")
    label_val = int(header.get(f"Segment{i}_LabelValue"))
    mask = data[i] == label_val
    masks[seg_id] = mask

# Volume data
volume_data = masks['lungs']

# Isosurface value
iso_value = 0.5

# Extract the isosurface
vertices, faces, normals, values = measure.marching_cubes(volume_data, iso_value)

edges = faces_to_edges(faces)

In [10]:
create_directory(f"{folder_path}/{file_name}")

In [14]:
with open(f"{folder_path}/{file_name}/vertices.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write each pair as a row in the CSV
    for vertex in vertices:
        writer.writerow(vertex)

In [16]:
with open(f"{folder_path}/{file_name}/edges.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    # Write each pair as a row in the CSV
    for edge in edges:
        writer.writerow(edge)